# Titanic simple model

In [2]:
from pandas.core.common import SettingWithCopyWarning
import warnings
from tensorflow import keras
import utils
import pandas as pd

warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [3]:
from loadDataUtils import loadDataUtils

In [4]:
path_train = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\train.csv'
path_test = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\test.csv'
data = loadDataUtils(path_train, path_test)

In [5]:
df_train, df_test = data.get_train_and_test()

### Clean data

In [6]:
from titanicPreprocessing import preprocess
import numpy as np

In [7]:
p = preprocess(df_train.copy(), df_test.copy())
p.do_preprocess()

In [8]:
train, test = p.get_data()

In [9]:
train_target = train['Survived']
train.drop(columns=['Survived'], inplace=True)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        889 non-null    int64  
 1   Pclass             889 non-null    int64  
 2   Age                889 non-null    float64
 3   SibSp              889 non-null    int64  
 4   Parch              889 non-null    int64  
 5   Fare               889 non-null    float64
 6   cabin_multiple     889 non-null    int64  
 7   Sex_female         889 non-null    uint8  
 8   Sex_male           889 non-null    uint8  
 9   Embarked_C         889 non-null    uint8  
 10  Embarked_Q         889 non-null    uint8  
 11  Embarked_S         889 non-null    uint8  
 12  cabin_letter_0     889 non-null    uint8  
 13  cabin_letter_A     889 non-null    uint8  
 14  cabin_letter_B     889 non-null    uint8  
 15  cabin_letter_C     889 non-null    uint8  
 16  cabin_letter_D     889 non

In [11]:
test.insert(20, 'cabin_letter_T', 0)

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        418 non-null    int64  
 1   Pclass             418 non-null    int64  
 2   Age                418 non-null    float64
 3   SibSp              418 non-null    int64  
 4   Parch              418 non-null    int64  
 5   Fare               418 non-null    float64
 6   cabin_multiple     418 non-null    int64  
 7   Sex_female         418 non-null    uint8  
 8   Sex_male           418 non-null    uint8  
 9   Embarked_C         418 non-null    uint8  
 10  Embarked_Q         418 non-null    uint8  
 11  Embarked_S         418 non-null    uint8  
 12  cabin_letter_0     418 non-null    uint8  
 13  cabin_letter_A     418 non-null    uint8  
 14  cabin_letter_B     418 non-null    uint8  
 15  cabin_letter_C     418 non-null    uint8  
 16  cabin_letter_D     418 non

## Drop not important columns

In [13]:
train.drop(columns=['Embarked_C'], inplace=True)
train.drop(columns=['Embarked_Q'], inplace=True)
train.drop(columns=['Embarked_S'], inplace=True)

train.drop(columns=['cabin_letter_0'], inplace=True)
train.drop(columns=['cabin_letter_A'], inplace=True)
train.drop(columns=['cabin_letter_B'], inplace=True)
train.drop(columns=['cabin_letter_C'], inplace=True)
train.drop(columns=['cabin_letter_D'], inplace=True)
train.drop(columns=['cabin_letter_E'], inplace=True)
train.drop(columns=['cabin_letter_F'], inplace=True)
train.drop(columns=['cabin_letter_G'], inplace=True)
train.drop(columns=['cabin_letter_T'], inplace=True)

test.drop(columns=['Embarked_C'], inplace=True)
test.drop(columns=['Embarked_Q'], inplace=True)
test.drop(columns=['Embarked_S'], inplace=True)

test.drop(columns=['cabin_letter_0'], inplace=True)
test.drop(columns=['cabin_letter_A'], inplace=True)
test.drop(columns=['cabin_letter_B'], inplace=True)
test.drop(columns=['cabin_letter_C'], inplace=True)
test.drop(columns=['cabin_letter_D'], inplace=True)
test.drop(columns=['cabin_letter_E'], inplace=True)
test.drop(columns=['cabin_letter_F'], inplace=True)
test.drop(columns=['cabin_letter_G'], inplace=True)
test.drop(columns=['cabin_letter_T'], inplace=True)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, train_target, test_size=0.3, random_state=42)

In [15]:
param_grid_lr = {'max_iter' : [2000],
              'penalty' : ['l1', 'l2'],
              'C' : np.logspace(-4, 4, 20),
              'solver' : ['liblinear']}

param_grid_knn = {'n_neighbors' : [3,5,7,9],
              'weights' : ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree','kd_tree'],
              'p' : [1,2]}

param_grid_svc = tuned_parameters = [{'kernel': ['rbf'], 'gamma': [.1,.5,1,2,5,10],
                                  'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['linear'], 'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['poly'], 'degree' : [2,3,4,5], 'C': [.1, 1, 10, 100, 1000]}]

param_grid_rf =  {'n_estimators': [100,500,1000],
                                  'bootstrap': [True,False],
                                  'max_depth': [3,5,10,20,50,75,100,None],
                                  'max_features': ['auto','sqrt'],
                                  'min_samples_leaf': [1,2,4,10],
                                  'min_samples_split': [2,5,10]}

param_grid_xgb = {
    'n_estimators': [20, 50, 100, 250, 500,1000],
    'colsample_bytree': [0.2, 0.5, 0.7, 0.8, 1],
    'max_depth': [2, 5, 10, 15, 20, 25, None],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2],
    'subsample': [0.5,0.6,0.7, 0.8, 0.9],
    'learning_rate':[.01,0.1,0.2,0.3,0.5, 0.7, 0.9],
    'gamma':[0,.01,.1,1,10,100],
    'min_child_weight':[0,.01,0.1,1,10,100],
    'sampling_method': ['uniform', 'gradient_based']
}

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.metrics import accuracy_score
from utils import clf_performance

In [17]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [18]:
svc_g = SVC(probability = True)
rf_g = RandomForestClassifier(random_state = 1)
xgb_g = XGBClassifier(random_state = 1)

In [19]:
clf_svc = GridSearchCV(svc_g, param_grid = param_grid_svc, cv = 5, verbose = True, n_jobs = -1)
best_clf_svc = clf_svc.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_svc,'SVC')

Fitting 5 folds for each of 55 candidates, totalling 275 fits
SVC
Best Score: 0.8392000000000002
Best Parameters: {'C': 10, 'degree': 3, 'kernel': 'poly'}


In [20]:
svc_best = SVC(C= 1, gamma= 0.5, kernel= 'rbf')
svc_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)

y_pred_best = svc_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(y_pred_best, y_test)

0.8052434456928839

In [21]:
clf_rf = GridSearchCV(rf_g, param_grid = param_grid_rf, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_rf,'Random Forest')

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits
Random Forest
Best Score: 0.8617677419354838
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_best = RandomForestClassifier(bootstrap= False, max_depth= 50, max_features= 'auto', min_samples_leaf= 2, min_samples_split= 10, n_estimators= 500)
rf_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
pred_rf_best = rf_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(pred_rf_best, y_test)

In [ ]:
clf_xgb = RandomizedSearchCV(xgb_g, param_grid = param_grid_xgb, cv = 5, verbose = True, n_jobs = -1)
best_clf_xgb = clf_xgb.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_xgb,'XGB')

In [ ]:
xgb_best = XGBClassifier(colsample_bytre= 0.75, gamma= 1, learning_rate= 0.5, max_depth= None, min_child_weight= 0.01, n_estimators= 450, reg_alpha= 1, reg_lambda= 5, sampling_method= 'uniform', subsample= 0.6)
xgb_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)

y_pred_best_xgb = xgb_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(y_pred_best_xgb, y_test)